In [ ]:
import requests
import csv
import dateparser
import collections

In [ ]:
## data is here
## https://docs.google.com/spreadsheets/d/1bMzc1YWJpO1Humx-owmdha-kI-aQ8EH83YJGiWMEv4U/edit?ts=6093d958#gid=868049820

data_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRJL-M4_V-uYdvw546V6VqvgF8eTMKBedrhjEyyXqSfWqed-6pU8N6K9SuhMi2SGcQ3rEx2uBoTjn7R/pub?gid=868049820&single=true&output=csv'
data = requests.get( data_url, stream=True )
data = list( csv.DictReader( data.content.decode('utf-8').splitlines() ) )

In [ ]:
start_dates = {}

In [ ]:
steps = collections.defaultdict( list )

for entry in data:
    
    try: 
        start, end = entry['step_dates'].split(' - ')

        start = dateparser.parse( start, settings={'DATE_ORDER': 'DMY'} )
        end = dateparser.parse( end, settings={'DATE_ORDER': 'DMY'} )

        step_length = (end - start).days
        
        steps[ entry['step_name'].lower() ].append( step_length )
        
        if entry['process'] not in start_dates:
            start_dates[ entry['process'] ] = start
        
    except:
        pass ## some steps are not complete days

In [ ]:
out = open('step_length.csv', 'w')

out.write('step,length\n')

for key, values in steps.items():
    
    for value in values:
        out.write( key + ',' + str(value) + '\n' )
        
out.close()

In [ ]:
## approximate starting date
for d in start_dates.values():
    print( d )

## Hidden steps

In [ ]:
import collections
import csv
import dateparser

In [ ]:
process = collections.defaultdict( list )

for entry in data:
    
    try: 
        start, end = entry['step_dates'].split(' - ')

        start = dateparser.parse( start, settings={'DATE_ORDER': 'DMY'} )
        end = dateparser.parse( end, settings={'DATE_ORDER': 'DMY'} )
        name = entry['step_name']
        
        process[ entry['process'] ].append( {
            'name': name,
            'start': start,
            'end': end
        })
            
    except:
        pass ## some steps are not complete days

In [ ]:
def compute_difference( step ):
    
    if step['end'] == None: ## manage cases where step does not have ending time
        return 0
    
    diff = step['end'] - step['start']
    
    return diff.days

In [ ]:
for instance, steps in process.items():
    
    print( instance )
    
    prev = steps[0]
    
    for step in steps[1:]:
        
        if not prev['end']:
            break ## sometimes we do not have exact ending times for steps
        
        gap = ( step['start'] - prev['end'] ).days
        
        if gap > 0:
            
            prev_diff = compute_difference( prev )
            step_diff = compute_difference( step )
            
            print( prev['name'], prev_diff , '>', step['name'], step_diff , '; gap', gap )
            
        prev = step